##This notebook aims at building transition function for GridWorld RL algorithm

In [17]:
import numpy as np
import random as rd
import decimal as dc
import fractions as fr
import sys
import copy
import pandas as pd

In [ ]:
# https://jermwatt.github.io/machine_learning_refined/notes/5_Linear_regression/5_6_Multi.html

In [32]:
class GridWorld(object):
    def __init__(self, dimension = 0, size=0):
        self.dimension =  dimension
        self.size= size
        if (self.dimension<=0):
          raise ValueError("1-La dimension doit être strictement positive.")
    
    def generate_d_empty_grid(self):   
        return([[(0,)*self.dimension]*self.size]*self.size)
    
    def generate_random_positions(self,Number_of_trials):
        nulls = [[0]*self.dimension]*self.dimension
        base_canonique = [[0]*self.dimension]
       
       #Je deteste faire des boucles, je n'ai pas eu de meilleure idée 
       #you're too high level for me i swear 
        for i in range(self.dimension):
            tampon_pos=copy.deepcopy(nulls[i])
            tampon_pos[i]=1
            tampon_neg=copy.deepcopy(nulls[i])
            tampon_neg[i]=-1
            base_canonique.append(tampon_pos)
            base_canonique.append(tampon_neg)
            #print(base_canonique)
            del(tampon_pos,tampon_neg)
        del(nulls)
        #faire tous les uplet de taille d dans 0,size  et tirer number of trials fois avec remise dans ca pour faire le truc d'entrainement 
        positions = [[rd.randint(0, self.size-1) for _ in range(self.dimension)] for _ in range(Number_of_trials)]
        def valid_moves(position,base_canonique):
            valid_movs = []
            for compteur in range(len(base_canonique)):
                tried_move= list(np.add(base_canonique[compteur],position))
                if (all(i < (self.size) and i>=0  for i in  tried_move)):
                   valid_movs.append(base_canonique[compteur])
              
            return(valid_movs)
        positions_and_associated_move=[]
        for i in positions:
          valid_movmnts = valid_moves(i,base_canonique)
          j=rd.choice(valid_movmnts)
          positions_and_associated_move.append([i,j,list(np.add(i,j))])
        df=pd.DataFrame(np.array([sum(positions_and_associated_move[i], []) for i in range(len(positions_and_associated_move))]), columns=["X_initial","Y_initial","move_X","move_Y","X_arriv","Y_arriv"])
        df["final"]=df[["X_arriv", "Y_arriv"]].values.tolist()
        df["depart"]=df[["X_initial", "Y_initial"]].values.tolist()
        return((df))
    



In [34]:
GW_2=GridWorld(2,9)


In [35]:
D_B = GW_2.generate_random_positions(10000)

In [37]:
np.array(D_B["depart"])

array([list([0, 6]), list([6, 4]), list([8, 1]), ..., list([8, 2]),
       list([3, 0]), list([6, 0])], dtype=object)

In [46]:
# linear regression for multioutput regression
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
X = D_B["depart"]

y=D_B["final"]

X=np.asarray([(X[i]) for i in range(len(X))])
y=np.asarray([(y[i]) for i in range(len(y))])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [92]:
X

array([[0, 6],
       [6, 4],
       [8, 1],
       ...,
       [8, 2],
       [3, 0],
       [6, 0]])

In [49]:
# define model
model = LinearRegression()
# fit model
model.fit(X_train, y_train)


LinearRegression()

In [98]:
Y_hat=model.predict(X_test)

In [59]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_hat, y_test)


0.3831764665020917

In [70]:
#quadratic regression 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
polyreg=make_pipeline(PolynomialFeatures(2),LinearRegression())
polyreg.fit(X_train,y_train)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [100]:
y_hat=polyreg.predict(X_test)
mean_squared_error(y_hat, y_test)
y_hat[1,1]
action=[[0,0]]*len(y_hat)
A=X_test-y_hat
a=map(round,A[1])
np.array(list(map(round, A[1])))
Y_hat

array([[3.02964842, 5.93581459],
       [6.87883835, 4.97646699],
       [3.99144186, 5.93784438],
       ...,
       [1.11009384, 3.9968215 ],
       [1.11211   , 3.02935475],
       [3.98942571, 6.90531113]])

In [97]:
y_test

array([[3, 7],
       [7, 6],
       [4, 5],
       ...,
       [2, 4],
       [1, 4],
       [3, 7]])

In [ ]:
#funny pattern: exemple the state is 3,7 it predicts 3,6 Or the state is 2,4 it predicts 1,4 

In [ ]:
#second learning problem: learn from a value S the grid configuration 
 